In [ ]:
# for SVM
from sklearn.decomposition import PCA # transform the dimestion to 2D data
from matplotlib.colors import ListedColormap
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
# define kernel function for sklearn.svm.SVC
def Linear_kernel(x, y):
  return np.dot(x, y.T)

In [ ]:
class SVM():
  def __init__(self, x, y, classes):
    self.classes = classes

    # dimension reduction
    pca = PCA(n_components=2)
    pca.fit(x)
    self.x = pca.transform(x)
    self.N = self.x.shape[0]

    # y need not be one-hot encoding
    self.Inverse_one_hot(y)
  
  # convert one-hot encoding to 1D
  def Inverse_one_hot(self, y):
    label = np.argmax(y, axis=1)
    self.y = np.array(label)

  # one-versus-the-rest
  def OvR(self, target_type):
    x1 = [self.x[c,:] for c in range(self.N) if self.y[c] == target_type]
    x2 = [self.x[c,:] for c in range(self.N) if self.y[c] != target_type]
    x = x1 + x2
    x = np.array(x)
    
    y1 = [1 for c in range(self.N) if self.y[c] == target_type]
    y2 = [-1 for c in range(self.N) if self.y[c] != target_type]
    y = y1 + y2
    y = np.array(y)
    
    return x, y

  # calculate w & b throught SVM
  def SVM_Process(self, x_train, y_train, target_type):
    # perform svm to get support vectors and Lagrange multipliers
    linear_svm = svm.SVC(kernel=Linear_kernel)
    linear_svm.fit(x_train, y_train)
    
    # get index of support vectors
    sv_index = linear_svm.support_

    # get the Lagrange multiplier (already multiply tn)
    alpha = np.abs(linear_svm.dual_coef_)
    
    # get support vector
    sv = [x_train[c,:] for c in range(x_train.shape[0]) if c in sv_index]
    sv = np.array(sv)
    
    # get the target value of support vectors
    t = [y_train[c] for c in range(y_train.shape[0]) if c in sv_index]
    t = np.array(t).reshape((1, -1))

    # target value reweighted
    alpha = alpha / t
    for i in range(t.shape[1]):
        if t[0,i] < 0:
            t[0,i] = -0.5
    alpha = alpha * t
    
    # calculate w
    w = np.dot(alpha, sv) # 1 x classes

    # calculate b
    b = 0
    for i, x1 in enumerate(sv):
        b += t[0,i]
        for j, x2 in enumerate(sv):
            b = b - alpha[0,j] * Linear_kernel(x1, x2)
    b = b / len(sv_index)
    
    sv_index = sv_index.tolist()
    
    return w, b
  
  # define voting criterion
  def Vote(self, x):
    # calculate score by w*x+b
    prediction = np.dot(x, self.w.T) + self.b
    vote = np.array(prediction)

    # choose the one that has the largest score
    label = np.argmax(vote, axis=1)

    return label

  # plot the geometry presentation
  def Plot(self, class_label):
    # define color for different classes
    colors = ['pink', 'wheat', 'red', 'yellow', 'lightgreen', 'lightblue', 'blue', 'purple', 'violet', 'silver']
    colors_dict = {0:'red', 1:'orange', 2:'darkred', 3:'gold', 4:'green', 5:'deepskyblue', 6:'navy', 7:'darkmagenta', 8:'darkviolet', 9:'dimgray'}
    my_map = ListedColormap(colors)
    
    # plot block color
    resolution = 100
    x1_range = np.linspace(np.min(self.x[:,0]), np.max(self.x[:,0]), resolution)
    x2_range = np.linspace(np.min(self.x[:,1]), np.max(self.x[:,1]), resolution)

    # use meshgrid to plot region color
    x1, x2 = np.meshgrid(x1_range, x2_range)

    # determine region color
    y_plot = []
    for i in x1_range:
      for j in x2_range:
        t = np.array([i, j]).reshape((1, 2))
        temp = self.Vote(t)
        y_plot.append(temp)
    
    y_plot = np.array(y_plot).reshape((x1.shape))
    # plot boundary
    plt.contourf(x1, x2, y_plot.T, cmap=my_map)
    
    # find support vectors
    linear_svm = svm.SVC(kernel=Linear_kernel)
    linear_svm.fit(self.x, self.y)
    sv = linear_svm.support_
    
    # plot the result
    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10 = None, None, None, None, None, None, None, None, None, None
    for i in range(self.N):
        class_type = int(self.label[i])
        if i in sv:
            pp = plt.scatter(self.x[i, 0], self.x[i, 1], marker='o', color=colors_dict[class_type], edgecolors='black')
        elif class_type == 0:
            p1 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[0])
        elif class_type == 1:
            p2 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[1])
        elif class_type == 2:
            p3 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[2])
        elif class_type == 3:
            p4 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[3])
        elif class_type == 4:
            p5 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[4])
        elif class_type == 5:
            p6 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[5])
        elif class_type == 6:
            p7 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[6])
        elif class_type == 7:
            p8 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[7])
        elif class_type == 8:
            p9 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[8])
        elif class_type == 9:
            p10 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[9])
        
    # construct the list for lenged
    p1 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[0])
    p2 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[1])
    p3 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[2])
    p4 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[3])
    p5 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[4])
    p6 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[5])
    p7 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[6])
    p8 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[7])
    p9 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[8])
    p10 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[9])
    # plot the first cv again to let the label right
    pp = plt.scatter(self.x[sv[0], 0], self.x[sv[0], 1], marker='o', facecolors='None', color='black')
    
    if self.classes == 2:
      plt.legend([p1, p2, pp], class_label, loc='upper left')
    if self.classes == 8:
      plt.legend([p1, p2, p3, p4, p5, p6, p7, p8, pp], class_label, loc='upper left')
    elif self.classes == 10:
      plt.legend([p1, p2, p3, p4, p5, p6, p7, p8, p9, pp], class_label, loc='upper left')
    
    plt.title('linear SVM')
    plt.show()
  
  def Truth(self, class_label):
    # define color for different classes#    colors = ['pink', 'wheat', 'red', 'yellow', 'lightgreen', 'lightblue', 'blue', 'purple', 'violet', 'silver']
    colors_dict = {0:'red', 1:'orange', 2:'darkred', 3:'gold', 4:'green', 5:'deepskyblue', 6:'navy', 7:'darkmagenta', 8:'darkviolet', 9:'dimgray'}
    my_map = ListedColormap(colors)
    
    # plot block color
    resolution = 100
    x1_range = np.linspace(np.min(self.x[:,0]), np.max(self.x[:,0]), resolution)
    x2_range = np.linspace(np.min(self.x[:,1]), np.max(self.x[:,1]), resolution)

    # use meshgrid to plot region color
    x1, x2 = np.meshgrid(x1_range, x2_range)

    # determine region color
    y_plot = []
    for i in x1_range:
      for j in x2_range:
        t = np.array([i, j]).reshape((1, 2))
        temp = self.Vote(t)
        y_plot.append(temp)
    
    y_plot = np.array(y_plot).reshape((x1.shape))
    # plot boundary
    plt.contourf(x1, x2, y_plot.T, cmap=my_map)
    
    # find support vectors
    linear_svm = svm.SVC(kernel=Linear_kernel)
    linear_svm.fit(self.x, self.y)
    sv = linear_svm.support_
    
    # plot the result
    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10 = None, None, None, None, None, None, None, None, None, None
    for i in range(self.N):
        class_type = int(self.y[i])
        if i in sv:
            pp = plt.scatter(self.x[i, 0], self.x[i, 1], marker='o', color=colors_dict[class_type], edgecolors='black')
        elif class_type == 0:
            p1 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[0])
        elif class_type == 1:
            p2 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[1])
        elif class_type == 2:
            p3 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[2])
        elif class_type == 3:
            p4 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[3])
        elif class_type == 4:
            p5 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[4])
        elif class_type == 5:
            p6 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[5])
        elif class_type == 6:
            p7 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[6])
        elif class_type == 7:
            p8 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[7])
        elif class_type == 8:
            p9 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[8])
        elif class_type == 9:
            p10 = plt.scatter(self.x[i, 0], self.x[i, 1], marker='x', color=colors_dict[9])
        
    # construct the list for lenged
    p1 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[0])
    p2 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[1])
    p3 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[2])
    p4 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[3])
    p5 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[4])
    p6 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[5])
    p7 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[6])
    p8 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[7])
    p9 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[8])
    p10 = plt.scatter(np.NaN, np.NaN, marker='x', color=colors_dict[9])
    # plot the first cv again to let the label right
    pp = plt.scatter(self.x[sv[0], 0], self.x[sv[0], 1], marker='o', facecolors='None', color='black')
    
    if self.classes == 2:
      plt.legend([p1, p2, pp], class_label, loc='upper left')
    if self.classes == 8:
      plt.legend([p1, p2, p3, p4, p5, p6, p7, p8, pp], class_label, loc='upper left')
    elif self.classes == 10:
      plt.legend([p1, p2, p3, p4, p5, p6, p7, p8, p9, pp], class_label, loc='upper left')
    
    plt.title('linear SVM')
    plt.show()

  # calculate predict accuracy
  def Precision(self):
    correct = [1 for i in range(self.N) if self.y[i] == self.label[i]]
    count = np.sum(correct)
    self.precision = count / self.N
    print('Presicion = '+str(round(self.precision, 2)))

  # main function of SVM
  def SVM(self):
    w, b = [], []

    # construct #(classes) SVM model
    for i in range(self.classes):
      # one-versus-the-rest
      x1, y1 = self.OvR(i)

      # perform svm to calculate w & b
      w1, b1 = self.SVM_Process(x1, y1, i)

      w.append(w1)
      b.append(b1)
    
    self.w = np.array(w).reshape(self.classes, 2)
    self.b = np.array(b)

    # predict the training data
    self.label = self.Vote(self.x)
    self.Precision()
  
  # use the sklearn package to perfrom SVM
  def SKLearn(self):
    linear_svm = svm.SVC(kernel='linear')
    linear_svm.fit(self.x, self.y)

    self.label = linear_svm.predict(self.x)
    self.w = linear_svm.coef_
    self.b = linear_svm.intercept_
    size = self.y.shape[0]
    print(accuracy_score(self.y.reshape((size, -1)), self.label.reshape((size, -1))))
    self.Precision()